# TPS 12 - Feature Engineering

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 3
TRAIN_SIZE = 500000

In [2]:
import numpy as np
import pandas as pd
import time
import os
import pyarrow
import gc

# Model/Evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [3]:
# Load full training data
train = pd.read_feather('../data/train.feather')

# Drop low/no variance 
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
train = train[train.Cover_Type != 5]

# Label Encoding
new_encoder = LabelEncoder()
train["Cover_Type"] = new_encoder.fit_transform(train["Cover_Type"])

# Split synthetic data
train, test = train_test_split(
    train, 
    train_size = TRAIN_SIZE, 
    random_state = RANDOM_SEED,
    stratify = train['Cover_Type'],
)
y_train = train['Cover_Type']


# features, data structure for summary scores
features = [x for x in train.columns if x not in ['Id','Cover_Type']]
nonsoil = [x for x in features if not x.startswith('Soil_Type')]
new_rows = list()
gc.collect()

print(f'Training Size: {train.shape[0]} rows, {train.shape[1]} cols')
print(f'Holdout Size: {test.shape[0]} rows, {test.shape[1]} cols\n')

Training Size: 500000 rows, 53 cols
Holdout Size: 3499999 rows, 53 cols



# Model 

In [4]:
# LightGBM Classifier
catboost_pipeline = make_pipeline(
    CatBoostClassifier(
        eval_metric = 'MultiClass',
        boosting_type = 'Plain',
        verbose = False,
        random_state = RANDOM_SEED,
    ),
)

# Scoring Function

In [5]:
def score_features(sklearn_model, processing = None):
    
    # Original Training/Test Split
    features = [x for x in train.columns if x not in ['Id','Cover_Type']]
    X_temp, X_test = train[features], test[features]
    y_temp, y_test = train['Cover_Type'], test['Cover_Type']
    
    # Feature Engineering
    if processing:
        X_temp = processing(X_temp)
        X_test = processing(X_test)
    features = [x for x in X_temp.columns]
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],6))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X_temp,y_temp)):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            n_repeats=10, random_state=RANDOM_SEED
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold} Accuracy:  {round(scores[fold], 5)} in {round(end-start,2)}s.')
        time.sleep(0.5)
    
    nonsoil = [x for x in X_test.columns if not x.startswith('Soil_Type')]
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    #print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    fi_scores = pd.Series(
        data = fi_scores, 
        index = features
    ).loc[nonsoil].sort_values()
    
    return scores.mean(), oof_preds, test_score, fi_scores

# Baseline

In [6]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline
)

new_rows.append((
    'Baseline', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95779 in 223.48s.
Fold 1 Accuracy:  0.95689 in 223.61s.
Fold 2 Accuracy:  0.95724 in 232.98s.
Train Accuracy: 0.95731
Test Accuracy: 0.95799
Training Time: 680.08s


Hillshade_9am                        -0.000052
Slope                                -0.000031
Hillshade_3pm                        -0.000026
Aspect                                0.000061
Wilderness_Area2                      0.000069
Hillshade_Noon                        0.000400
Wilderness_Area4                      0.001785
Wilderness_Area1                      0.006181
Horizontal_Distance_To_Hydrology      0.010465
Vertical_Distance_To_Hydrology        0.017887
Wilderness_Area3                      0.022897
Horizontal_Distance_To_Fire_Points    0.030866
Horizontal_Distance_To_Roadways       0.047241
Elevation                             0.461235
dtype: float64

# Feature Engineering

## 1 . Fix Aspect and Hillshade Values

In [7]:
# Fix aspect
def fix_ranges(data):
    
    df = data.copy()
    
    # Fix Aspect
    df["Aspect"][df["Aspect"] < 0] += 360
    df["Aspect"][df["Aspect"] > 359] -= 360
    
    # Fix Hillshade
    df.loc[df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
    df.loc[df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
    df.loc[df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
    df.loc[df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
    df.loc[df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
    df.loc[df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
    
    return df

In [8]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    fix_ranges
)

new_rows.append((
    'Fix_Range', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.9579 in 229.5s.
Fold 1 Accuracy:  0.95747 in 205.15s.
Fold 2 Accuracy:  0.95735 in 229.73s.
Train Accuracy: 0.95757
Test Accuracy: 0.95809
Training Time: 664.38s


Slope                                -0.000047
Hillshade_9am                        -0.000007
Aspect                                0.000012
Hillshade_3pm                         0.000034
Wilderness_Area2                      0.000060
Hillshade_Noon                        0.000478
Wilderness_Area4                      0.001913
Wilderness_Area1                      0.006201
Horizontal_Distance_To_Hydrology      0.010727
Vertical_Distance_To_Hydrology        0.018090
Wilderness_Area3                      0.023078
Horizontal_Distance_To_Fire_Points    0.031052
Horizontal_Distance_To_Roadways       0.047457
Elevation                             0.461452
dtype: float64

## 2. Water Distance Features

In [9]:
# Distance to Water
def water_distance_features(data):
    df = data.copy()
    
    # use float64 for squaring
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float64')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float64')
    
    # compute metrics
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    
    # convert back
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float32')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float32')
    df["Hydro_Taxicab"] = df["Hydro_Taxicab"].astype('float32')
    df["Hydro_Euclid"] = df["Hydro_Euclid"].astype('float32')
    
    return df

In [10]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    water_distance_features
)

new_rows.append((
    'Water_Dist', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95769 in 183.92s.
Fold 1 Accuracy:  0.95714 in 182.43s.
Fold 2 Accuracy:  0.95772 in 182.95s.
Train Accuracy: 0.95752
Test Accuracy: 0.958
Training Time: 549.29s


Hillshade_3pm                        -0.000040
Slope                                 0.000005
Hillshade_9am                         0.000036
Aspect                                0.000067
Wilderness_Area2                      0.000097
Hillshade_Noon                        0.000407
Horizontal_Distance_To_Hydrology      0.000658
Hydro_Euclid                          0.000907
Wilderness_Area4                      0.001433
Hydro_Taxicab                         0.005329
Wilderness_Area1                      0.005928
Vertical_Distance_To_Hydrology        0.014037
Wilderness_Area3                      0.023241
Horizontal_Distance_To_Fire_Points    0.031064
Horizontal_Distance_To_Roadways       0.047479
Elevation                             0.461855
dtype: float64

## 3. Count Features

In [11]:
# Create count features
def count_features(data):
    
    df = data.copy()
    soil_features = [x for x in df.columns if x.startswith("Soil_Type")]
    wilderness_features = [x for x in df.columns if x.startswith("Wilderness_Area")]

    # Count features
    df["Soil_Count"] = df[soil_features].apply(sum, axis=1)
    df["Wilderness_Count"] = df[wilderness_features].apply(sum, axis=1)
    
    return df

In [12]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    count_features
)

new_rows.append((
    'Count', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95953 in 239.32s.
Fold 1 Accuracy:  0.95906 in 239.34s.
Fold 2 Accuracy:  0.95886 in 222.28s.
Train Accuracy: 0.95915
Test Accuracy: 0.95948
Training Time: 700.94s


Hillshade_9am                         0.000034
Wilderness_Count                      0.000046
Hillshade_3pm                         0.000059
Aspect                                0.000067
Wilderness_Area2                      0.000081
Slope                                 0.000093
Hillshade_Noon                        0.000547
Wilderness_Area4                      0.001440
Wilderness_Area1                      0.004639
Horizontal_Distance_To_Hydrology      0.010794
Vertical_Distance_To_Hydrology        0.018301
Wilderness_Area3                      0.024470
Horizontal_Distance_To_Fire_Points    0.031983
Soil_Count                            0.034981
Horizontal_Distance_To_Roadways       0.048675
Elevation                             0.462630
dtype: float64

## 4. Shade Features

In [13]:
# Create Shade features
def new_shade_features(data):
    
    df = data.copy()
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

    # Hillshade
    df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    
    return df

In [14]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    new_shade_features
)

new_rows.append((
    'Shade_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95763 in 183.45s.
Fold 1 Accuracy:  0.95717 in 183.04s.
Fold 2 Accuracy:  0.95744 in 182.89s.
Train Accuracy: 0.95741
Test Accuracy: 0.95789
Training Time: 549.39s


Hillshade_Avg                        -0.000013
Slope                                 0.000014
Hillshade_Range                       0.000021
Wilderness_Area2                      0.000032
Hillshade_3pm                         0.000046
Hillshade_9am                         0.000062
Aspect                                0.000114
Hillshade_Noon                        0.000480
Wilderness_Area4                      0.001522
Wilderness_Area1                      0.005699
Horizontal_Distance_To_Hydrology      0.010593
Vertical_Distance_To_Hydrology        0.017933
Wilderness_Area3                      0.023351
Horizontal_Distance_To_Fire_Points    0.030998
Horizontal_Distance_To_Roadways       0.047392
Elevation                             0.461632
dtype: float64

## 5. Distance Interactions

In [15]:
def distance_interactions(data):
    
    df = data.copy()
    df['Hydro_Fire_1'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    return df

In [16]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    distance_interactions
)

new_rows.append((
    'Dist_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95802 in 196.47s.
Fold 1 Accuracy:  0.95752 in 195.81s.
Fold 2 Accuracy:  0.95768 in 195.2s.
Train Accuracy: 0.95774
Test Accuracy: 0.95839
Training Time: 587.48s


Hillshade_3pm                        -0.000023
Hillshade_9am                         0.000003
Slope                                 0.000010
Aspect                                0.000066
Wilderness_Area2                      0.000069
Hillshade_Noon                        0.000487
Hydro_Road_1                          0.000856
Wilderness_Area4                      0.002443
Horizontal_Distance_To_Fire_Points    0.002878
Fire_Road_2                           0.003479
Hydro_Road_2                          0.004264
Fire_Road_1                           0.004372
Hydro_Fire_2                          0.004838
Hydro_Fire_1                          0.005138
Wilderness_Area1                      0.005502
Horizontal_Distance_To_Hydrology      0.006956
Vertical_Distance_To_Hydrology        0.017869
Horizontal_Distance_To_Roadways       0.022260
Wilderness_Area3                      0.023560
Elevation                             0.461088
dtype: float64

## 6. Elevation Interactions

In [17]:
def elevation_interactions(data):
    
    df = data.copy()
    df['EHiElv'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['EViElv'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['EVDtH'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['EHDtH'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    
    # Throwaway Features most likely
    df['Highwater'] = (df.Vertical_Distance_To_Hydrology < 0).astype(int)
    df['Hillshade_3pm_is_zero'] = (df.Hillshade_3pm == 0).astype(int)
    
    return df

In [18]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    elevation_interactions
)

new_rows.append((
    'Elev_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95856 in 193.85s.
Fold 1 Accuracy:  0.95793 in 191.18s.
Fold 2 Accuracy:  0.95825 in 191.29s.
Train Accuracy: 0.95825
Test Accuracy: 0.95877
Training Time: 576.32s


Hillshade_3pm                        -5.359992e-05
EHiElv                               -5.199865e-06
EViElv                               -5.400007e-11
Hillshade_3pm_is_zero                 2.000008e-07
Highwater                             3.200022e-06
Slope                                 3.140008e-05
Wilderness_Area2                      4.359999e-05
Hillshade_9am                         5.320020e-05
Aspect                                8.900002e-05
Hillshade_Noon                        3.781999e-04
Vertical_Distance_To_Hydrology        4.140002e-04
Wilderness_Area4                      2.456600e-03
Horizontal_Distance_To_Hydrology      4.834200e-03
Wilderness_Area1                      5.928797e-03
Wilderness_Area3                      2.321340e-02
Horizontal_Distance_To_Fire_Points    3.131400e-02
EHDtH                                 4.338862e-02
Horizontal_Distance_To_Roadways       4.700640e-02
Elevation                             1.365212e-01
EVDtH                          

# Summary

In [19]:
pd.DataFrame.from_records(
    data = new_rows,
    columns = ['features','cv_scores','holdout','recall_0', 'recall_1','recall_2','recall_3','recall_4','recall_5']
).sort_values('holdout')

,features,cv_scores,holdout,recall_0,recall_1,recall_2,recall_3,recall_4,recall_5
4,Shade_Features,0.957412,0.957893,0.964101,0.972026,0.875041,0.106383,0.400560,0.634974
0,Baseline,0.957308,0.957987,0.963900,0.971874,0.873610,0.148936,0.409664,0.641141
2,Water_Dist,0.957518,0.957996,0.964118,0.971945,0.875572,0.170213,0.414566,0.639728
1,Fix_Range,0.957572,0.958091,0.964101,0.972040,0.875164,0.148936,0.414566,0.641526
5,Dist_Interactions,0.957742,0.958390,0.964565,0.972479,0.871035,0.106383,0.413866,0.638957
6,Elev_Interactions,0.958250,0.958773,0.964641,0.973087,0.874387,0.127660,0.413165,0.637158
3,Count,0.959152,0.959480,0.964526,0.971923,0.882317,0.127660,0.441877,0.709881
